In [1]:
import numpy as np
print(np)
%matplotlib nbagg
import matplotlib
from scipy import special

<module 'numpy' from '/Users/susanclark/dedalus_upgraded2/dedalus/lib/python3.5/site-packages/numpy/__init__.py'>


/Users/susanclark/dedalus_upgraded2/dedalus/lib/python3.5/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import dedalus.public as de
import numpy as np
import matplotlib.pylab as plt

In [3]:
nr1 = 64
r_shortgrid = de.Chebyshev('r', nr1, interval=(1, 2))
d1 = de.Domain([r_shortgrid])

nr2 = 92
r_longgrid = de.Chebyshev('r', nr2, interval=(1, 2))
d2 = de.Domain([r_longgrid])

In [4]:
# critical parameters
r1 = 1.
r2 = 2.

mu_omega = 0.27 # = Omega2/Omega1
eta_omega = r1/r2

c1 = (mu_omega - eta_omega**2)/(1 - eta_omega**2)
c2 = ((1 - mu_omega)/(1 - eta_omega**2))#*r1**2

# Hollerbach & Rudiger 2005
Pm = 1.0E-6
Re = 1521
xi = 4
Q = 2.33
k = Q

iR = 1.0/Re
Rm = Pm*Re
iRm = 1.0/Rm

Ha = 16.3
beta = (2*Re*Rm)/(Ha**2)

magnetic_bcs = "insulating"
Pmzero = False
#magnetic_bcs = "conducting"

In [5]:
widegap1 = de.EVP(d1,['psi','u', 'A', 'B', 'psir', 'psirr', 'psirrr', 'ur', 'Ar', 'Br'],'sigma')
widegap2 = de.EVP(d2,['psi','u', 'A', 'B', 'psir', 'psirr', 'psirrr', 'ur', 'Ar', 'Br'],'sigma')

for widegap in [widegap1, widegap2]:
    widegap.parameters['k'] = Q
    widegap.parameters['Re'] = Re
    widegap.parameters['Rm'] = Rm
    widegap.parameters['Hasq'] = Ha**2
    widegap.parameters['c1'] = c1
    widegap.parameters['c2'] = c2
    widegap.parameters['B0'] = 1
    widegap.parameters['xi'] = xi
    widegap.parameters['r1'] = r1
    widegap.parameters['r2'] = r2
    #widegap.parameters['bessel1'] = special.iv
    widegap.parameters['bessel1'] = special.iv(0, k*r1)/special.iv(1, k*r1)
    widegap.parameters['bessel2'] = special.kn(0, k*r2)/special.kn(1, k*r2)
    
    widegap.substitutions['ru0'] = '(r*r*c1 + c2)' # u0 = r Omega(r) = Ar + B/r
    widegap.substitutions['rrdu0'] = '(c1*r*r-c2)' # du0/dr = A - B/r^2
    widegap.substitutions['dz'] = '(1j*k)'
    widegap.substitutions['r4D4psi'] = '(r**4*dr(psirrr) + r**4*dz**4*psi + r**4*2*dz**2*psirr + 3*r*psir - 3*r**2*psirr + 2*r**3*psirrr + 2*r**3*dz**2*psir - 2*r**2*dz**2*psi - 3*psi)'
    widegap.substitutions['r4D2psi'] = '(r**4*psirr + r**4*dz**2*psi + r**3*psir - r**2*psi)'
    widegap.substitutions['r4D2A'] = '(r**4*dr(Ar) + r**4*dz**2*A + r**3*Ar - r**2*A)'
    widegap.substitutions['r2D2u'] = '(r**2*dr(ur) + r**2*dz**2*u + r*ur - u)'
    widegap.substitutions['r2D2A'] = '(r**2*dr(Ar) + r**2*dz**2*A + r*Ar - A)' 
    widegap.substitutions['r2D2B'] = '(r**2*dr(Br) + r**2*dz**2*B + r*Br - B)'
    
    # psi multiplied by r**4, others by r**2
    widegap.add_equation("Re*sigma*r4D2psi - r4D4psi + Re*2*r**2*ru0*dz*u - Hasq*dz*(r4D2A + 2*xi*r**2*B) = 0")
    widegap.add_equation("Re*sigma*r**2*u - r2D2u - Re*2*c1*dz*psi - Hasq*dz*B = 0") 
    if Pmzero is True:
        widegap.add_equation("-r2D2A - r**2*dz*psi = 0")
        widegap.add_equation("-r2D2B - r**2*dz*u + 2*xi*dz*psi = 0") 
    else:
        widegap.add_equation("Rm*sigma*r**2*A - r2D2A - r**2*dz*psi = 0")
        widegap.add_equation("Rm*sigma*r**2*B - r2D2B - Rm*2*c2*dz*A - r**2*dz*u + 2*xi*dz*psi = 0") 
    

    widegap.add_equation("dr(psi) - psir = 0")
    widegap.add_equation("dr(psir) - psirr = 0")
    widegap.add_equation("dr(psirr) - psirrr = 0")
    widegap.add_equation("dr(u) - ur = 0")
    widegap.add_equation("dr(A) - Ar = 0")
    widegap.add_equation("dr(B) - Br = 0")
    
    widegap.add_bc('left(u) = 0')
    widegap.add_bc('right(u) = 0')
    widegap.add_bc('left(psi) = 0')
    widegap.add_bc('right(psi) = 0')
    widegap.add_bc('left(psi + r*psir) = 0')
    widegap.add_bc('right(psi + r*psir) = 0')
    if magnetic_bcs == "conducting":
        widegap.add_bc('left(A) = 0')
        widegap.add_bc('right(A) = 0')
        widegap.add_bc('left(B + r*Br) = 0')
        widegap.add_bc('right(B + r*Br) = 0')

    if magnetic_bcs == "insulating":
        widegap.add_bc('left(dr(r*dz*A) - k*r*bessel1*dz*A) = 0')
        widegap.add_bc('right(dr(r*dz*A) + k*r*bessel2*dz*A) = 0')
        widegap.add_bc('left(B) = 0')
        widegap.add_bc('right(B) = 0')


2016-06-17 14:32:42,524 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2016-06-17 14:32:42,525 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10


In [ ]:
widegap_solver1 = widegap1.build_solver()
widegap_solver2 = widegap2.build_solver()

In [ ]:
widegap_solver1.solve(widegap_solver1.pencils[0])
widegap_solver2.solve(widegap_solver2.pencils[0])

/Users/susanclark/dedalus_upgraded2/dedalus/lib/python3.5/site-packages/scipy/linalg/decomp.py:56: RuntimeWarning: divide by zero encountered in true_divide
  w = alpha / beta


In [ ]:
ev1 = widegap_solver1.eigenvalues
ev2 = widegap_solver2.eigenvalues

In [ ]:
def discard_spurious_eigenvalues(lambda1, lambda2):
    
    """
    lambda1 :: eigenvalues from low res run
    lambda2 :: eigenvalues from high res run

    Solves the linear eigenvalue problem for two different resolutions.
    Returns trustworthy eigenvalues using nearest delta, from Boyd chapter 7.
    """

    # Reverse engineer correct indices to make unsorted list from sorted
    reverse_lambda1_indx = np.arange(len(lambda1)) 
    reverse_lambda2_indx = np.arange(len(lambda2))

    lambda1_and_indx = np.asarray(list(zip(lambda1, reverse_lambda1_indx)))
    lambda2_and_indx = np.asarray(list(zip(lambda2, reverse_lambda2_indx)))

    # remove nans
    lambda1_and_indx = lambda1_and_indx[np.isfinite(lambda1)]
    lambda2_and_indx = lambda2_and_indx[np.isfinite(lambda2)]

    # Sort lambda1 and lambda2 by real parts
    lambda1_and_indx = lambda1_and_indx[np.argsort(lambda1_and_indx[:, 0].real)]
    lambda2_and_indx = lambda2_and_indx[np.argsort(lambda2_and_indx[:, 0].real)]

    lambda1_sorted = lambda1_and_indx[:, 0]
    lambda2_sorted = lambda2_and_indx[:, 0]

    # Compute sigmas from lower resolution run (gridnum = N1)
    sigmas = np.zeros(len(lambda1_sorted))
    sigmas[0] = np.abs(lambda1_sorted[0] - lambda1_sorted[1])
    sigmas[1:-1] = [0.5*(np.abs(lambda1_sorted[j] - lambda1_sorted[j - 1]) + np.abs(lambda1_sorted[j + 1] - lambda1_sorted[j])) for j in range(1, len(lambda1_sorted) - 1)]
    sigmas[-1] = np.abs(lambda1_sorted[-2] - lambda1_sorted[-1])

    if not (np.isfinite(sigmas)).all():
        print("WARNING: at least one eigenvalue spacings (sigmas) is non-finite (np.inf or np.nan)!")

    # Nearest delta
    delta_near = np.array([np.nanmin(np.abs(lambda1_sorted[j] - lambda2_sorted)/sigmas[j]) for j in range(len(lambda1_sorted))])

    # Discard eigenvalues with 1/delta_near < 10^6
    lambda1_and_indx = lambda1_and_indx[np.where((1.0/delta_near) > 1E6)]
    #print(lambda1_and_indx)

    lambda1 = lambda1_and_indx[:, 0]
    indx = lambda1_and_indx[:, 1]

    return lambda1, indx

In [ ]:
goodeigs, goodeigs_indices = discard_spurious_eigenvalues(ev1, ev2)

In [ ]:
def ploteigs(goodevals):

    """
    Plot real vs imaginary parts of eigenvalues.
    """
    
    # Color is sign of imaginary part
    colors = ["blue" for i in range(len(goodevals))]
    imagpos = np.where(goodevals.imag >= 0)
    for p in imagpos[0]:
        colors[p] = "red"
  
    # Symbol is sign of real part
    symbols = ["." for i in range(len(goodevals))]
    thickness = np.zeros(len(goodevals))
    realpos = np.where(goodevals.real >= 0)
    for p in realpos[0]:
        symbols[p] = "+"
        thickness[p] = 2
    
    print("Number of positive real parts", len(realpos[0]))
    
    for x, y, c, s, t in zip(np.abs(goodevals.real), np.abs(goodevals.imag), colors, symbols, thickness):
        plt.plot(x, y, s, c=c, alpha = 0.5, ms = 8, mew = t)
        
    # Dummy plot for legend
    plt.plot(0, 0, '+', c = "red", alpha = 0.5, mew = 2, label = r"$\gamma \geq 0$, $\omega > 0$")
    plt.plot(0, 0, '+', c = "blue", alpha = 0.5, mew = 2, label = r"$\gamma \geq 0$, $\omega < 0$")
    plt.plot(0, 0, '.', c = "red", alpha = 0.5, label = r"$\gamma < 0$, $\omega > 0$")
    plt.plot(0, 0, '.', c = "blue", alpha = 0.5, label = r"$\gamma < 0$, $\omega < 0$")
        
    plt.legend()
        
    plt.loglog()
    plt.xlabel(r"$\left|\gamma\right|$", size = 15)
    plt.ylabel(r"$\left|\omega\right|$", size = 15, rotation = 0)
    plt.title(r"$\mathrm{HMRI}$ $\mathrm{eigenvalues}$ $\lambda = \gamma + i \omega$")

In [ ]:
print("There are {} goodeigs".format(len(goodeigs)))
print(goodeigs)
plt.figure()
ploteigs(goodeigs)

In [ ]:
marginal_mode_index = np.where(np.abs(goodeigs.real) == np.nanmin(np.abs(goodeigs.real)))[0][0]
use_indx = int(goodeigs_indices[marginal_mode_index])

print(goodeigs[marginal_mode_index])
print(goodeigs[goodeigs.real > 0])
print(goodeigs[goodeigs.imag == np.max(goodeigs.imag)])

In [ ]:
rr  = r_shortgrid.grid()
evector = widegap_solver1.eigenvectors[use_indx]
widegap_solver1.set_state(use_indx)
data = widegap_solver1.state

In [ ]:
plt.figure()
plt.plot(rr, (data['psi']['g']).real, label='real')
plt.plot(rr, (data['psi']['g']).imag, label='imag')

In [ ]:
plt.figure()
plt.plot(rr, (data['u']['g']).real, label='real')
plt.plot(rr, (data['u']['g']).imag, label='imag')

In [ ]:
plt.figure()
plt.plot(rr, (data['A']['g']).real, label='real')
plt.plot(rr, (data['A']['g']).imag, label='imag')

In [ ]:
plt.figure()
plt.plot(rr, (data['B']['g']).real, label='real')
plt.plot(rr, (data['B']['g']).imag, label='imag')